Import req libraries

In [287]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import shutil

Create folder for output images

In [288]:
path = "GeneratedImages"
exist = os.path.exists(path)
if not exist:
    os.makedirs(path)
    print("Directoty Created")
else:
    print('Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    shutil.rmtree(path)
    os.makedirs(path)

Directory Exists


Check folder with images of frames to modify is present

In [289]:
path = "Starting"
exist = os.path.exists(path)
if not exist:
    os.makedirs(path)
    print("Directoty Created")
else:
    print('Directory Exists')

Directory Exists


Check there are frames in folder, and create array of all files

In [290]:
files = [f for f in listdir(path) if isfile(join(path, f))]
files.remove('startingImages.zip')

In [291]:
files

['dayShadows.png', 'dayLowishLight.png', 'day1.jpg', 'day2.png']

In [292]:
for file in files:
    print(file)
    #dont want the '.jpg' element of the file name in the file name
    noExtension = file[:-4]

    #load image
    img = cv2.imread('Starting/'+file)
    xA = 802
    xB = 942
    yA = 903
    yB = 1043

    #crop to steering wheel logo
    crop = img[xA:xB, yA:yB]

    grayImage = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                
    (thresh, contrastImg) = cv2.threshold(grayImage, 100, 255, cv2.THRESH_BINARY)

    backtorgb = cv2.cvtColor(contrastImg,cv2.COLOR_GRAY2RGB)


    #save crop
    cv2.imwrite(f'GeneratedImages/{noExtension}_0_degrees_x0y0_crop.jpg',backtorgb)
    
    #array of all angle values to rotate by
    angles = np.linspace(-90,90,240)
    
    #set pixel shift
    shiftMax= 4
    shiftMin = -4
    
    for i in range(shiftMin,shiftMax):
        for j in range(shiftMin,shiftMax):
            for a in angles:
                angle = round(a)

                #finder center of image
                (h, w) = img.shape[:2]
                (cX, cY) = (w // 2, h // 2)
                
                # rotate our image by angle a
                M = cv2.getRotationMatrix2D((971, 894), angle, 1.0)
                rotated = cv2.warpAffine(img, M, (w, h))
                
                #crop to steering wheel logo
                x1 = xA + i
                x2 = xB + i
                y1 = yA + j
                y2 = yB + j
                crop = rotated[x1:x2, y1:y2]

                grayImage = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                
                (thresh, contrastImg) = cv2.threshold(grayImage, 100, 255, cv2.THRESH_BINARY)

                backtorgb = cv2.cvtColor(contrastImg,cv2.COLOR_GRAY2RGB)

                # ground truth
                # scale angle between 0 and 1
                angle = angle + 90
                angle = round((angle-(0))/(180-(0)),2)

                #save image
                cv2.imwrite(f'GeneratedImages/{noExtension}_{angle}_degrees_x{i}y{j}_crop.jpg',backtorgb)                   

dayShadows.png
dayLowishLight.png
day1.jpg
day2.png
